# Adding nodes to the graph
## Adding 2 cast nodes to the first graph
# ![Graph 1](resources/Graph_1.jpg)


In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto
import numpy as np
# The protobuf definition can be found here:
# https://github.com/onnx/onnx/blob/master/onnx/onnx.proto


# Create inputs (ValueInfoProto)
X1 = helper.make_tensor_value_info('X1', TensorProto.FLOAT, [1, 1])
X2 = helper.make_tensor_value_info('X2', TensorProto.FLOAT, [1, 1])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 1])

# Creating a Node (NodeProto)
node_def = helper.make_node(
    'Mul', # node name
    ['X1', 'X2'], # inputs
    ['Y'], # outputs
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    'test-model',
    [X1, X2],
    [Y],
)

# # Creating the model (ModelProto)
# model_def = helper.make_model(graph_def, producer_name='onnx-example')

# # Check and Save the model
# onnx.checker.check_model(model_def)
# onnx.save(model_def, 'Models/onnx_mutate.onnx')

## Creating the 2 nodes to be added to the graph

In [2]:
# Create Input (ValueInforProto)
I1 = helper.make_tensor_value_info('I1', TensorProto.STRING, [1, 1])
I2 = helper.make_tensor_value_info('I2', TensorProto.STRING, [1, 1])

# Create output (ValueInfoProto)
X1 = helper.make_tensor_value_info('X1', TensorProto.FLOAT, [1, 1])
X2 = helper.make_tensor_value_info('X2', TensorProto.FLOAT, [1, 1])

node_def_1 = helper.make_node(
    'Cast',
    ['I1'], # inputs
    ['X1'], # outputs
    name='c1',
    to=getattr(TensorProto, 'FLOAT'),
)

node_def_2 = helper.make_node(
    'Cast',
    ['I2'], # inputs
    ['X2'], # outputs
    name='c2',
    to=getattr(TensorProto, 'FLOAT'),
)


## Adding the nodes to the model
The nodes can be added to the `GraphProto`. Depending on the location of the nodes in the graph, they can be either added to the graph input or graph output.
The example here adds to `Cast` nodes to the starting of the graph hence `graph.input` is used.

In [3]:
# Adding the required nodes at the topological sorted order
graph_def.node.insert(0,node_def_1)
graph_def.node.insert(1,node_def_2)

# Adjusting the input parameters of the graph
graph_def.input.pop()
graph_def.input.pop()
graph_def.input.insert(0, I1)
graph_def.input.insert(1, I2)

In [4]:
print(graph_def)

node {
  input: "I1"
  output: "X1"
  name: "c1"
  op_type: "Cast"
  attribute {
    name: "to"
    i: 1
    type: INT
  }
}
node {
  input: "I2"
  output: "X2"
  name: "c2"
  op_type: "Cast"
  attribute {
    name: "to"
    i: 1
    type: INT
  }
}
node {
  input: "X1"
  input: "X2"
  output: "Y"
  op_type: "Mul"
}
name: "test-model"
input {
  name: "I1"
  type {
    tensor_type {
      elem_type: 8
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "I2"
  type {
    tensor_type {
      elem_type: 8
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 1
        }
      }
    }
  }
}
output {
  name: "Y"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 1
        }
      }
    }
  }
}



In [5]:
# Creating the model(ModelProto) and checking the same
model_def = helper.make_model(graph_def, producer_name='onnx-add-node-example')
onnx.checker.check_model(model_def)